In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px
from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
symbols_path = './backbone/data/backtest/symbols/SP500_H1.csv'
df = pd.read_csv(symbols_path)
df

,Date,Open,High,Low,Close,Volume
0,2012-01-16 00:00:00,1290.900,1291.100,1288.100,1288.100,0.0000
1,2012-01-16 01:00:00,1287.900,1288.400,1286.400,1287.400,0.0000
2,2012-01-16 02:00:00,1287.600,1287.600,1286.600,1287.100,0.0000
3,2012-01-16 03:00:00,1286.900,1288.400,1286.900,1287.100,0.0000
4,2012-01-16 04:00:00,1286.900,1287.100,1286.100,1286.100,0.0000
...,...,...,...,...,...,...
63251,2024-08-30 16:00:00,5600.337,5605.937,5581.014,5600.354,29.2410
63252,2024-08-30 17:00:00,5600.116,5607.377,5595.127,5597.849,20.0450
63253,2024-08-30 18:00:00,5597.970,5618.452,5595.220,5616.233,14.5300
63254,2024-08-30 19:00:00,5615.983,5651.087,5610.517,5645.139,23.4720


In [4]:
# from datetime import datetime
# import MetaTrader5 as mt5
# import pytz

# print("MetaTrader5 package author: ", mt5.__author__)
# print("MetaTrader5 package version: ", mt5.__version__)

# # establish connection to MetaTrader 5 terminal
# if not mt5.initialize():
#     raise Exception("initialize() failed, error code =",mt5.last_error())

# # set time zone to UTC
# timezone = pytz.timezone("Etc/UTC")

# # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
# utc_from = datetime(2013, 5, 1, tzinfo=timezone)
# utc_to = datetime(2024, 1, 1, tzinfo=timezone)
# rates = mt5.copy_rates_range('USDCAD', mt5.TIMEFRAME_H1, utc_from, utc_to)

# mt5.shutdown()

# # create DataFrame out of the obtained data
# df = pd.DataFrame(rates)

# # convert time in seconds into the datetime format
# df['time'] = pd.to_datetime(df['time'], unit='s')
                          
# df = df.rename(columns={
#   'time':'Date', 
#   'open':'Open', 
#   'high':'High', 
#   'low':'Low', 
#   'close':'Close', 
#   'tick_volume':'Volume'
# })

# df

In [5]:
train_start = '2014-01-01'
train_end = '2016-01-01'

test_start = '2015-01-01'
test_end = '2018-01-01'

wfo_start = '2014-01-01'
wfo_end = '2023-01-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]
wfo_data = df[(df.Date > wfo_start) & (df.Date < wfo_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
wfo_data.loc[:, 'Date'] = pd.to_datetime(wfo_data.Date)

df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')
wfo_data = wfo_data.set_index('Date')

df = df.set_index('Date')


In [9]:

import itertools

class LowerLow(Strategy):
    pip_size = 0.0001
    risk=2
    n=10
    day_to_buy = 25
    
    def init(self):
        self.daily_sma_200 = resample_apply(
            'D', ta.SMA, self.data.Close, timeperiod=200
        )

        self.rsi_2 = self.I(
            ta.RSI, self.data.Close, 2
        )
        

    def next(self):
        actual_close = self.data.Close[-1]
        today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')

        if self.position:
            first_trade = self.trades[0]
            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if time_in_position > 5:
                    self.position.close()

        else:
            if today.day == self.day_to_buy:
                self.buy(size=self.risk/100)
                self.random_time = 5




# metrics = pd.DataFrame()

# n_bars = [3, 4, 5, 8, 10]
# qs = [25, 50, 75]
# bars_percentile = [30, 60, 90]

# parameter_combinations = list(itertools.product(
#     n_bars, qs, bars_percentile
# ))

# for n, q, bar_percentile in parameter_combinations:
#     bt_train = Backtest(
#         train_data, 
#         BreakoutTwist,
#         commission=0.0002, 
#         cash=15_000, 
#         margin=1/30
#     )

#     stats = bt_train.run(
#         n=n,
#         q=q,
#         bars_percentile=bar_percentile
#     )

#     # bt_train.plot(filename='./RsiBBands.html', resample=False)

#     win_rate = stats['Win Rate [%]']
#     avg_trade_perc = stats['Avg. Trade [%]']

#     actual_metrics = pd.DataFrame(
#         {
#             'winning_rate':[win_rate], 
#             'avg_trade_perc':[avg_trade_perc], 
#             'n_candles':[n], 
#             'q':[q], 
#             'bars_percentile':[bar_percentile], 
#         }
#     )

#     metrics = pd.concat([metrics, actual_metrics])

# print(stats)
# metrics.sort_values(by='winning_rate', ascending=False)

bt_train = Backtest(
    train_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run(
)
stats

Start                     2014-01-01 12:00:00
End                       2015-12-31 17:00:00
Duration                    729 days 05:00:00
Exposure Time [%]                    7.937128
Equity Final [$]                  14697.32064
Equity Peak [$]                       15000.0
Return [%]                          -2.017862
Buy & Hold Return [%]               11.321139
Return (Ann.) [%]                    -0.91151
Volatility (Ann.) [%]                2.790149
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.676294
Avg. Drawdown [%]                   -4.676294
Max. Drawdown Duration      462 days 17:00:00
Avg. Drawdown Duration      462 days 17:00:00
# Trades                                   10
Win Rate [%]                             20.0
Best Trade [%]                       3.577074
Worst Trade [%]                     -1.785553
Avg. Trade [%]                    

In [10]:
bt_train = Backtest(
    test_data, 
    LowerLow,
    commission=0.0002, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2015-01-02 00:00:00
End                       2017-12-29 20:00:00
Duration                   1092 days 20:00:00
Exposure Time [%]                    9.120386
Equity Final [$]                 15350.555835
Equity Peak [$]                  15376.955835
Return [%]                           2.337039
Buy & Hold Return [%]               29.597007
Return (Ann.) [%]                    0.744358
Volatility (Ann.) [%]                  1.7872
Sharpe Ratio                         0.416494
Sortino Ratio                        0.674545
Calmar Ratio                          0.34381
Max. Drawdown [%]                   -2.165026
Avg. Drawdown [%]                   -0.409259
Max. Drawdown Duration      546 days 15:00:00
Avg. Drawdown Duration       55 days 02:00:00
# Trades                                   17
Win Rate [%]                        64.705882
Best Trade [%]                       2.716447
Worst Trade [%]                     -1.154307
Avg. Trade [%]                    

## Test Exit: Monkey Entry

In [ ]:

trade_frecuency = stats['# Trades'] / train_data.shape[0]
long_frecuency = stats._trades[stats._trades['Size'] > 0].shape[0] / stats['# Trades']
short_frecuency = stats._trades[stats._trades['Size'] < 0].shape[0] / stats['# Trades']

def random_boolean(prob_true=0.5):
    return random.choices([True, False], weights=[prob_true, 1 - prob_true], k=1)[0]


class VwapExit(Strategy):
    pip_size = 0.0001
    sl_pips = 15
    rr = 1.5
    risk = 1

    adx_period = 5
    rsi_period = 3

    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, 14)

        self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close = close_prices[-1]
       
        if self.position:
            if self.position.is_long and self.rsi > 70:
                self.position.close()
                
            elif self.position.is_short and self.rsi < 30:
                self.position.close()

        else: 
            trade = random_boolean(prob_true=trade_frecuency)

            if trade:
                long = random_boolean(prob_true=long_frecuency)

                if long:
                    sl = actual_close - self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.buy(sl=sl, size=units)

                else:
                    sl = actual_close + self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    
                    # Calculate lot size in units
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.sell(sl=sl, size=units)


metrics = pd.DataFrame()

for x in range(0, 10):
    bt_train = Backtest(
        train_data, 
        VwapExit, 
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run(
    )

    # bt_train.plot(filename='./RsiBBands.html', resample=False)

    equity = stats['Equity Final [$]']
    return_ = stats['Return [%]']
    sharpe_ratio = stats['Sharpe Ratio']

    actual_metrics = pd.DataFrame(
        {
            'equity':[equity], 
            'return_':[return_], 
            'sharpe_ratio':[sharpe_ratio], 
        }
    )

    metrics = pd.concat([metrics, actual_metrics])



In [ ]:
metrics.return_.std()

## Full system

In [ ]:
import itertools

class VwapRsiFull(Strategy):
    pip_size = 0.0001
    sl_pips = 10
    rr = 1
    risk = 1

    n_candles = 7 
    distance = 1

    rsi_up_threshold=70
    rsi_down_threshold=30


    def init(self):
        self.vwap = self.I(
            pandas_ta.vwap, 
            pd.Series(self.data.High, index=self.data.index),
            pd.Series(self.data.Low, index=self.data.index),
            pd.Series(self.data.Close, index=self.data.index),
            pd.Series(self.data.Volume, index=self.data.index),
        ) 

        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=200) 
        
        self.rsi = self.I(ta.RSI, self.data.Close, 14)


    def next(self):
        actual_close = self.data.Close[-1]
       
        if self.position:
            pass

        else: 

            actual_vwap = self.vwap[-1]
            distance_vwap = diff_pips(actual_vwap, actual_close, pip_value=self.pip_size, absolute=True)

            n_candles_under_vwap = True
            n_candles_up_vwap = True

            for x in range(1, self.n_candles):
                if self.data.Close[-x] > self.vwap[-x]:
                    n_candles_under_vwap = False

                if self.data.Close[-x] < self.vwap[-x]:
                    n_candles_up_vwap = False

            if distance_vwap <= self.distance and n_candles_up_vwap: 
                sl = actual_close - self.sl_pips * self.pip_size
                tp = actual_close + self.rr * self.sl_pips * self.pip_size
                
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.buy(sl=sl, size=units, tp=tp)

            elif distance_vwap <= self.distance and n_candles_under_vwap: 
                sl = actual_close + self.sl_pips * self.pip_size
                tp = actual_close - self.rr * self.sl_pips * self.pip_size

                account_risk = self.equity * (self.risk / 100)
                
                # Calculate lot size in units
                units = round(account_risk / (self.pip_size * self.sl_pips))
                self.sell(sl=sl, size=units, tp=tp)


bt_train = Backtest(
    train_data, 
    VwapRsiFull, 
    commission=0.0002,
    cash=15_000, 
    margin=1/30
)


stats = bt_train.run(rr=2, sl_pips=15)

# stats = bt_train.optimize(
#     distance=[1, 2, 3],
#     n_candles=[6, 8,  10, 12],
#     sl_pips=[5, 8, 12, 15],
#     rsi_up_threshold=[60, 70, 80],
#     rsi_down_threshold=[20, 30, 40],
# )

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


In [ ]:
stats._strategy.__dict__

# WFO

In [ ]:
import pickle
from wfo_utils.utils import walk_forward, plot_full_equity_curve, plot_stats

In [ ]:

lookback_bars = 365 * 24 + 200 * 24
validation_bars = 180 * 24
warmup_bars = 200 * 24 

params = {
    'n': [10],
    'maximize': 'Return [%]' 
}


stats = walk_forward(
    LowerLow,
    wfo_data, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

In [ ]:
for x in range(0, len(stats)):
    plot_stats(df, stats[x], LowerLow, plot=True)

In [ ]:
plot_full_equity_curve(
    df, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
